EDA

In [94]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import auxilary.lists as aux_lists

In [27]:
# applicattion_test=pl.read_csv('data/application_test.csv')
application_train=pl.read_csv('data/application_train.csv')
buereau_balance=pl.read_csv('data/bureau_balance.csv')
buereau=pl.read_csv('data/bureau.csv')
installments_payments=pl.read_csv('data/installments_payments.csv')
POS_CASH_balance=pl.read_csv('data/POS_CASH_balance.csv')
previous_application=pl.read_csv('data/previous_application.csv')

In [50]:
from catboost import CatBoostClassifier

In [26]:
application_train['TARGET'].value_counts()

TARGET,counts
i64,u32
0,282686
1,24825


In [ ]:
ownership_cols=[]

In [56]:
string_cols=application_train.select(pl.col(pl.Utf8)).columns

### Feature importance

In [57]:
cat=CatBoostClassifier(cat_features=string_cols)

In [89]:
training_sample=application_train.sample(30000,shuffle=True,seed=1)
for col in string_cols:
    training_sample=training_sample.with_columns(pl.col(col).fill_null('None').alias(col))
for col in [col for col in training_sample.columns if col not in string_cols]:
    training_sample=training_sample.with_columns(pl.col(col).fill_null(-1).alias(col))
X_sample=training_sample.drop(columns=['TARGET','SK_ID_CURR'])
y_sample=training_sample['TARGET']

In [91]:
cat.fit(X_sample.to_pandas(),y_sample.to_pandas(),verbose=0)
preds=cat.predict(X_sample.to_pandas())

In [92]:
print(classification_report(y_sample,preds))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     27474
           1       0.95      0.12      0.21      2526

    accuracy                           0.93     30000
   macro avg       0.94      0.56      0.58     30000
weighted avg       0.93      0.93      0.90     30000



In [93]:
pl.DataFrame(
    {
        "feature": X_sample.columns,
        "importance": cat.feature_importances_,
    }
).sort('importance',descending=True)[:20]

feature,importance
str,f64
"""EXT_SOURCE_3""",9.774466
"""EXT_SOURCE_2""",9.095577
"""EXT_SOURCE_1""",4.902448
"""DAYS_BIRTH""",4.430119
"""AMT_CREDIT""",3.836847
"""AMT_GOODS_PRIC…",3.678498
"""AMT_ANNUITY""",3.429203
"""DAYS_REGISTRAT…",3.406614
"""DAYS_EMPLOYED""",3.353197


### Col cats

### Common Sense Removal

In [49]:
application_train.filter(pl.col('FLAG_OWN_CAR')=="Y")['OWN_CAR_AGE'].null_count()

5

In [ ]:
cols_to_remove=[]
cols_to_remove.append('FLAG_OWN_CAR')

In [37]:
application_train.head()[:,20:]

DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,…,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
-2120,null,1,1,0,1,1,0,"""Laborers""",1.0,2,2,"""WEDNESDAY""",10,0,0,0,0,0,0,"""Business Entit…",0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.0,0.069,0.0833,0.125,0.0369,0.0202,0.019,0.0,…,0.0,"""reg oper accou…","""block of flats…",0.0149,"""Stone, brick""","""No""",2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
-291,null,1,1,0,1,1,0,"""Core staff""",2.0,1,1,"""MONDAY""",11,0,0,0,0,0,0,"""School""",0.311267,0.622246,null,0.0959,0.0529,0.9851,0.796,0.0605,0.08,0.0345,0.2917,0.3333,0.013,0.0773,0.0549,0.0039,…,0.01,"""reg oper accou…","""block of flats…",0.0714,"""Block""","""No""",1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
-2531,26.0,1,1,1,1,1,0,"""Laborers""",1.0,2,2,"""MONDAY""",9,0,0,0,0,0,0,"""Government""",null,0.555912,0.729567,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
-2437,null,1,1,0,1,0,0,"""Laborers""",2.0,2,2,"""WEDNESDAY""",17,0,0,0,0,0,0,"""Business Entit…",null,0.650442,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null
-3458,null,1,1,0,1,0,0,"""Core staff""",1.0,2,2,"""THURSDAY""",11,0,0,0,0,1,1,"""Religion""",null,0.322738,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
application_train.null_count()

SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,…,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,12,278,1292,0,0,0,0,0,0,0,0,0,202929,0,0,0,0,0,0,96391,2,0,0,0,0,0,0,0,…,169682,210295,154297,148431,156341,145755,1021,1021,1021,1021,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41519,41519,41519,41519,41519,41519


In [33]:
[col for col in previous_application.columns if col not in application_train.columns]

['SK_ID_PREV',
 'AMT_APPLICATION',
 'AMT_DOWN_PAYMENT',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NFLAG_LAST_APPL_IN_DAY',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'DAYS_DECISION',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'SELLERPLACE_AREA',
 'NAME_SELLER_INDUSTRY',
 'CNT_PAYMENT',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'NFLAG_INSURED_ON_APPROVAL']

In [21]:
previous_application

SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
i64,i64,str,f64,f64,f64,f64,f64,str,i64,str,i64,f64,f64,f64,str,str,i64,str,str,str,str,str,str,str,str,i64,str,f64,str,str,f64,f64,f64,f64,f64,f64
2030495,271877,"""Consumer loans…",1730.43,17145.0,17145.0,0.0,17145.0,"""SATURDAY""",15,"""Y""",1,0.0,0.182832,0.867336,"""XAP""","""Approved""",-73,"""Cash through t…","""XAP""",null,"""Repeater""","""Mobile""","""POS""","""XNA""","""Country-wide""",35,"""Connectivity""",12.0,"""middle""","""POS mobile wit…",365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,"""Cash loans""",25188.615,607500.0,679671.0,null,607500.0,"""THURSDAY""",11,"""Y""",1,null,null,null,"""XNA""","""Approved""",-164,"""XNA""","""XAP""","""Unaccompanied""","""Repeater""","""XNA""","""Cash""","""x-sell""","""Contact center…",-1,"""XNA""",36.0,"""low_action""","""Cash X-Sell: l…",365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,"""Cash loans""",15060.735,112500.0,136444.5,null,112500.0,"""TUESDAY""",11,"""Y""",1,null,null,null,"""XNA""","""Approved""",-301,"""Cash through t…","""XAP""","""Spouse, partne…","""Repeater""","""XNA""","""Cash""","""x-sell""","""Credit and cas…",-1,"""XNA""",12.0,"""high""","""Cash X-Sell: h…",365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,"""Cash loans""",47041.335,450000.0,470790.0,null,450000.0,"""MONDAY""",7,"""Y""",1,null,null,null,"""XNA""","""Approved""",-512,"""Cash through t…","""XAP""",null,"""Repeater""","""XNA""","""Cash""","""x-sell""","""Credit and cas…",-1,"""XNA""",12.0,"""middle""","""Cash X-Sell: m…",365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,"""Cash loans""",31924.395,337500.0,404055.0,null,337500.0,"""THURSDAY""",9,"""Y""",1,null,null,null,"""Repairs""","""Refused""",-781,"""Cash through t…","""HC""",null,"""Repeater""","""XNA""","""Cash""","""walk-in""","""Credit and cas…",-1,"""XNA""",24.0,"""high""","""Cash Street: h…",null,null,null,null,null,null
1383531,199383,"""Cash loans""",23703.93,315000.0,340573.5,null,315000.0,"""SATURDAY""",8,"""Y""",1,null,null,null,"""Everyday expen…","""Approved""",-684,"""Cash through t…","""XAP""","""Family""","""Repeater""","""XNA""","""Cash""","""x-sell""","""Credit and cas…",-1,"""XNA""",18.0,"""low_normal""","""Cash X-Sell: l…",365243.0,-654.0,-144.0,-144.0,-137.0,1.0
2315218,175704,"""Cash loans""",null,0.0,0.0,null,null,"""TUESDAY""",11,"""Y""",1,null,null,null,"""XNA""","""Canceled""",-14,"""XNA""","""XAP""",null,"""Repeater""","""XNA""","""XNA""","""XNA""","""Credit and cas…",-1,"""XNA""",null,"""XNA""","""Cash""",null,null,null,null,null,null
1656711,296299,"""Cash loans""",null,0.0,0.0,null,null,"""MONDAY""",7,"""Y""",1,null,null,null,"""XNA""","""Canceled""",-21,"""XNA""","""XAP""",null,"""Repeater""","""XNA""","""XNA""","""XNA""","""Credit and cas…",-1,"""XNA""",null,"""XNA""","""Cash""",null,null,null,null,null,null
2367563,342292,"""Cash loans""",null,0.0,0.0,null,null,"""MONDAY""",15,"""Y""",1,null,null,null,"""XNA""","""Canceled""",-386,"""XNA""","""XAP""",null,"""Repeater""","""XNA""","""XNA""","""XNA""","""Credit and cas…",-1,"""XNA""",null,"""XNA""","""Cash""",null,null,null,null,null,null
